In [1]:
import sys
old_stdout = sys.stdout
sys.stdout = open('log2.txt', 'w')

In [2]:
import os
import tempfile
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Bidirectional, 
    LSTM, 
    GlobalAveragePooling1D, 
    Dense, 
    Input, 
    concatenate,
    TimeDistributed,
    Flatten
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [3]:
train_dir = 'data/train'
test_dir = 'data/test'
# 비디오 파일 목록과 태그를 포함하는 리스트를 만드는 함수
def create_data_list(data_dir):
    data_list = []
    # data_dir 안의 각 디렉토리에 대해 반복
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)  # 아이템의 전체 경로
        # 해당 경로가 디렉토리인지 확인
        if os.path.isdir(item_path):
            # 디렉토리 내의 모든 파일을 나열
            for file_name in os.listdir(item_path):
                # 파일이 .mp4 파일인지 확인
                if file_name.endswith('.mp4'):
                    # 리스트에 태그와 파일 경로를 추가
                    data_list.append((item, str(data_dir+'/'+item)+'/'+file_name))
    return data_list

# 함수를 사용해서 리스트를 생성
train_list = create_data_list(train_dir)
test_list = create_data_list(test_dir)
# 리스트에서 데이터프레임을 생성
train_df = pd.DataFrame(data=train_list, columns=['tag', 'video_name'])
test_df = pd.DataFrame(data=test_list, columns=['tag', 'video_name'])
# 필요한 경우 열 순서를 수정
train_df = train_df.loc[:, ['tag', 'video_name']]
test_df = test_df.loc[:, ['tag', 'video_name']]
# 데이터프레임을 CSV 파일로 저장
train_file_path = 'train.csv'
test_file_path = 'test.csv'
train_df.to_csv(train_file_path, encoding='utf-8-sig', index=False)
test_df.to_csv(test_file_path, encoding='utf-8-sig', index=False)

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print(f"Total video for training: {len(train_df)}")
print(f"Total video for testing: {len(test_df)}")

In [5]:
# GPU 장치 목록 가져오기
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    # GPU의 가상 메모리 제한을 6GB로 설정
    tf.config.experimental.set_virtual_device_configuration(
        gpu_devices[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 6)]
    )
    # set_memory_growth는 set_virtual_device_configuration과 함께 사용할 수 없습니다
else:
    print('Using CPU')

In [6]:
IMG_SIZE = 500
BATCH_SIZE = 32
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
SKELETON_FEATURES = 33*4
HAND_FEATURES = 21*3*2

In [8]:
# 주어진 이미지에서 중앙에 맞춰 정사각형으로 잘나내는 함수
def crop_center_square(frame):
    # 이미지의 높이(y)와 너비(x)를 가져옴
    y, x = frame.shape[0:2]
    # 이미지의 높이와 너비 중 더 작은 값을 선택하여 정사각형의 크기를 결정
    min_dim = min(y, x)
    # 정사각형을 이미지 중앙에 위치시키기 위해 시작점의 x좌표와 y좌표를 계산
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    # 계산된 시작점과 정사각형의 크기를 이용하여 이미지의 중앙 부분을 잘라냅니다.
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [9]:
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])  # 첫 번째 점
    b = np.array([b.x, b.y])  # 중간 점 (팔꿈치)
    c = np.array([c.x, c.y])  # 세 번째 점 (손목)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [10]:
# 비디오 파일을 로드하고, 각 프레임을 처리하여 배열로 반환하는 함수
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    mp_hands = mp.solutions.hands
    mp_pose = mp.solutions.pose

    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, smooth_landmarks=True)
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    # OpenCV를 사용하여 비디오 파일 열기
    cap = cv2.VideoCapture(path)
    frames = []
    skeletons = []  # 스켈레톤 데이터
    hand_landmarks = []  # 손 데이터
    arm_angles = []  # 팔 각도 데이터

    try:
        while True:
            # 비디오에서 프레임을 하나씩 읽기
            ret, frame = cap.read()
            # 읽을 프레임이 없으면 반복문을 종료
            if not ret:
                break
            # 읽은 프레임에서 중앙의 정사각형 부분을 잘라냄
            frame = crop_center_square(frame)
            # 프레임의 크기를 지정된 크기로 조절
            frame = cv2.resize(frame, resize)            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Mediapipe를 사용하여 스켈레톤 추출
            hands_results = hands.process(frame_rgb)
            pose_results = pose.process(frame_rgb)
           
            if pose_results.pose_landmarks:
                right_shoulder = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                right_elbow = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
                right_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

                left_shoulder = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                left_elbow = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
                left_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

                right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

                arm_angles.append((right_arm_angle, left_arm_angle))
                skeletons.append(pose_results.pose_landmarks.landmark)
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)            
                
            if hands_results.multi_hand_landmarks:
                hand_landmarks_data = hands_results.multi_hand_landmarks
                hand_landmarks.append(hand_landmarks_data)
                for hand_lm in hand_landmarks_data:
                    mp.solutions.drawing_utils.draw_landmarks(
                        frame, hand_lm, mp_hands.HAND_CONNECTIONS)
            
            cv2.imshow('Video Frame', frame)
            cv2.waitKey(30)
            # OpenCV는 BGR 색상 순서를 사용하므로, 이를 RGB 순서로 변경
            frame = frame[:, :, [2, 1, 0]]
            # 처리된 프레임을 프레임 리스트에 추가
            frames.append(frame)
            # max_frames가 지정된 경우, 지정된 수의 프레임만큼만 처리
            if len(frames) == max_frames:
                break
    finally:
        # 비디오 파일을 닫기
        cv2.destroyAllWindows()
        cap.release()
        pose.close
        hands.close
    return np.array(frames), skeletons, hand_landmarks, np.array(arm_angles)

In [11]:
# 특징추출
def build_feature_extractor():
    # 이미지 특징 추출을 위한 InceptionV3 모델
    base_model = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input
    image_input = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed_image = preprocess_input(image_input)
    image_features = base_model(preprocessed_image)

    # Mediapipe 데이터 처리
    mediapipe_input = keras.Input((258,))
    mediapipe_features = keras.layers.Dense(258, activation="relu")(mediapipe_input)
    mediapipe_features = keras.layers.Dropout(0.5)(mediapipe_features)  # Dropout 추가

    # 팔 각도 데이터 처리
    arm_angle_input = keras.Input((2,))
    arm_angle_features = keras.layers.Dense(16, activation="relu")(arm_angle_input)
    arm_angle_features = keras.layers.Dropout(0.5)(arm_angle_features)  # Dropout 추가

    # 데이터 결합 및 추가 처리
    combined_features = keras.layers.concatenate([image_features, mediapipe_features, arm_angle_features])
    combined_features = keras.layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01))(combined_features)  # 정규화 추가

    # 최종 출력 레이어
    outputs = keras.layers.Dense(5, activation="softmax")(combined_features)  # 클래스 수에 맞게 조정

    return keras.Model(inputs=[image_input, mediapipe_input, arm_angle_input], outputs=outputs, name="feature_extractor")

In [12]:
# 손 랜드마크를 2개로 제한한 코드
def preprocess_skeleton_data(skeleton):
    # 스켈레톤 데이터가 없는 경우 빈 벡터 반환
    if not skeleton:
        return np.zeros(SKELETON_FEATURES)
    # 스켈레톤 데이터를 1차원 배열로 변환
    skeleton_array = np.array([[lm.x, lm.y, lm.z] for lm in skeleton]).flatten()    
    # 부족한 부분을 0으로 채우기
    skeleton_array = np.pad(skeleton_array, ((0, max(0, SKELETON_FEATURES - len(skeleton_array)))))    
    return skeleton_array

def preprocess_hand_data(hand_landmarks):
    # 손 랜드마크 데이터가 없는 경우 빈 벡터 반환
    if not hand_landmarks or len(hand_landmarks) < 2:
        return np.zeros(HAND_FEATURES)
    
    # 첫 번째와 두 번째 손에 대한 랜드마크만 처리
    hand_data = []
    for hand_lm in hand_landmarks[:2]:  # 첫 번째와 두 번째 손만 처리
        lm_array = np.array([[lm.x, lm.y, lm.z] for lm in hand_lm.landmark]).flatten()
        hand_data.extend(lm_array)

    # 부족한 부분을 0으로 채우기
    hand_data = np.pad(hand_data, ((0, max(0, HAND_FEATURES - len(hand_data)))))

    return np.array(hand_data)

def preprocess_arm_angle_data(arm_angles):
    # 팔 각도 데이터가 없는 경우 빈 벡터 반환
    if not arm_angles:
        return np.zeros(2)  # 오른팔, 왼팔 각각의 각도
    return np.array(arm_angles)

def preprocess_image(frame):    
    frame = image.img_to_array(frame[0])  # frame[0]으로 변경
    frame = preprocess_input(frame)  # ResNet50의 전처리 함수를 사용하여 정규화
    return frame

In [13]:
def prepare_all_video(df):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    # Mediapipe 데이터를 저장할 배열 초기화                          
    frame_skeletons = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, SKELETON_FEATURES), dtype="float16")
    frame_hands = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, HAND_FEATURES), dtype="float16")
    
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float16")
    
    # 이미지 데이터 저장할 배열 초기화
    frame_images = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3), dtype="float16")
    # 팔 각도 데이터 저장할 배열 초기화
    frame_arm_angles = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, 2), dtype="float16")  # 2는 팔 각도 데이터 차원
    
    # 특징 추출기 모델 초기화
    feature_extractor = build_feature_extractor()

    for idx, path in enumerate(video_paths):
        frames, skeletons, hands, arm_angles = load_video(path)
        video_length = min(MAX_SEQ_LENGTH, len(frames))

        for i in range(video_length):
            # 이미지 데이터 전처리 및 특징 추출
            image_feature = preprocess_image(frames[i:i+1])
            image_feature = np.expand_dims(image_feature, axis=0)  # 차원 확장
                      
            # Mediapipe 데이터 전처리
            skeleton_feature = preprocess_skeleton_data(skeletons[i])
            hand_feature = preprocess_hand_data(hands[i])
            combined_mediapipe_data = np.concatenate([skeleton_feature, hand_feature])
            combined_mediapipe_data = np.expand_dims(combined_mediapipe_data, axis=0)
            
            # 팔 각도 데이터 전처리
            arm_angle_feature = np.array(arm_angles[i])
            arm_angle_feature = np.expand_dims(arm_angle_feature, axis=0)
            
            # 모델 예측
            try:
                # feature_extractor에 모든 데이터 전달
                frame_feature = feature_extractor.predict([image_feature, combined_mediapipe_data, arm_angle_feature], verbose=0)
                frame_features[idx, i, :] = frame_feature  # 데이터 저장
            except Exception as e:
                print("Error during prediction:", e)
            
            # 데이터 저장
            frame_images[idx, i, :] = frames[i]  # 원본 이미지 데이터 저장
            frame_skeletons[idx, i, :] = skeleton_feature
            frame_hands[idx, i, :] = hand_feature
            frame_arm_angles[idx, i, :] = arm_angle_feature
            frame_masks[idx, i] = 1
            
    # 반환 값에 Mediapipe 데이터 포함
    return (frame_features, frame_skeletons, frame_hands, frame_arm_angles, frame_masks)

In [14]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO 메시지를 숨긴다.
tf.get_logger().setLevel('ERROR')         # TensorFlow 로그를 ERROR 레벨로 설정한다.

In [76]:
train_data = prepare_all_video(train_df)

label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
train_labels = train_df["tag"].values
train_labels = label_processor(labels[..., None]).numpy()
train_labels = np.squeeze(train_labels)
sys.stdout = old_stdout

Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into shape (2048,)
Error during prediction: could not broadcast input array from shape (5,) into sh

KeyboardInterrupt: 

In [ ]:
test_data = prepare_all_video(test_df)
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(test_df["tag"]))
test_labels = test_df["tag"].values
test_labels = label_processor(labels[..., None]).numpy()
test_labels = np.squeeze(test_labels)
sys.stdout = old_stdout

In [68]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()
    
    # 기존 이미지 특징에 대한 입력
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    skeleton_input = keras.Input((MAX_SEQ_LENGTH, SKELETON_FEATURES))
    hand_input = keras.Input((MAX_SEQ_LENGTH, HAND_FEATURES))
    arm_angle_input = keras.Input((MAX_SEQ_LENGTH, 2))  # 팔 각도 데이터 입력 레이어
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")
    
    # LSTM 레이어 처리
    x = LSTM(64, return_sequences=True)(frame_features_input, mask=mask_input)
    x = LSTM(32, return_sequences=True)(x)
    x = GlobalAveragePooling1D()(x)
    
    # Mediapipe 데이터 처리
    y_skeleton = GlobalAveragePooling1D()(skeleton_input)
    y_hand = GlobalAveragePooling1D()(hand_input)

    # 팔 각도 데이터 처리
    y_arm_angle = GlobalAveragePooling1D()(arm_angle_input)

    # 데이터 결합
    combined = concatenate([x, y_skeleton, y_hand, y_arm_angle])

    # 추가 처리
    z = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.01))(combined)
    output = Dense(len(class_vocab), activation="softmax", kernel_regularizer=regularizers.l2(0.01))(z)
    
    # 모델 생성 및 컴파일
    lstm_model = keras.Model([frame_features_input, skeleton_input, hand_input, arm_angle_input, mask_input], output)
    lstm_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return lstm_model

In [70]:
EPOCHS = 200
BATCH_SIZE= 128
def run_experiment():
    filepath = "tmp/video_classifier_lstm.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1], train_data[2],train_data[3], train_data[4]],
        train_labels,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    
    _, accuracy = seq_model.evaluate(
        [test_data[0], test_data[1], test_data[2], test_data[3], test_data[4]],
        test_labels
    )
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    # 손실 및 정확도 그래프 출력
    plt.figure(figsize=(10, 4))
    # 손실 그래프
    ax1 = plt.subplot(1, 1, 1)
    ax1.plot(history.history['loss'], label='Train Loss', color='blue')
    ax1.plot(history.history['val_loss'], label='Validation Loss', color='blue', linestyle='dashed')
    ax1.set_title('Training and Validation Loss and Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.legend(loc='upper left')
    # 정확도 그래프를 같은 그래프에 추가
    ax2 = ax1.twinx()
    ax2.plot(history.history['accuracy'], label='Train Accuracy', color='green')
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', color='green', linestyle='dashed')
    ax2.set_ylabel('Accuracy', color='green')
    ax2.tick_params(axis='y', labelcolor='green')
    ax2.legend(loc='upper right')
    plt.show()
    
    seq_model.save('test_lstm_model2.h5')
    
    return history, seq_model

_, sequence_model = run_experiment()

Epoch 1/200
1/1 [==============================] - ETA: 0s - loss: 27.6522 - accuracy: 0.2564
Epoch 1: val_loss improved from inf to 19.74641, saving model to tmp\video_classifier_lstm.h5
1/1 [==============================] - 12s 12s/step - loss: 27.6522 - accuracy: 0.2564 - val_loss: 19.7464 - val_accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - ETA: 0s - loss: 26.5577 - accuracy: 0.2564
Epoch 2: val_loss improved from 19.74641 to 18.66704, saving model to tmp\video_classifier_lstm.h5
1/1 [==============================] - 0s 81ms/step - loss: 26.5577 - accuracy: 0.2564 - val_loss: 18.6670 - val_accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - ETA: 0s - loss: 25.4716 - accuracy: 0.2564
Epoch 3: val_loss improved from 18.66704 to 17.58889, saving model to tmp\video_classifier_lstm.h5
1/1 [==============================] - 0s 82ms/step - loss: 25.4716 - accuracy: 0.2564 - val_loss: 17.5889 - val_accuracy: 0.0000e+00
Epoch 4/200
1/1 [=======

1/1 [==============================] - ETA: 0s - loss: 7.5837 - accuracy: 0.2564
Epoch 26: val_loss improved from 2.84370 to 2.74284, saving model to tmp\video_classifier_lstm.h5
1/1 [==============================] - 0s 78ms/step - loss: 7.5837 - accuracy: 0.2564 - val_loss: 2.7428 - val_accuracy: 0.0000e+00
Epoch 27/200
1/1 [==============================] - ETA: 0s - loss: 7.2163 - accuracy: 0.2564
Epoch 27: val_loss improved from 2.74284 to 2.63130, saving model to tmp\video_classifier_lstm.h5
1/1 [==============================] - 0s 79ms/step - loss: 7.2163 - accuracy: 0.2564 - val_loss: 2.6313 - val_accuracy: 0.0000e+00
Epoch 28/200
1/1 [==============================] - ETA: 0s - loss: 6.8152 - accuracy: 0.2564
Epoch 28: val_loss improved from 2.63130 to 2.51356, saving model to tmp\video_classifier_lstm.h5
1/1 [==============================] - 0s 76ms/step - loss: 6.8152 - accuracy: 0.2564 - val_loss: 2.5136 - val_accuracy: 0.0000e+00
Epoch 29/200
1/1 [=======================

1/1 [==============================] - 0s 61ms/step - loss: 2.0143 - accuracy: 0.2308 - val_loss: 4.5574 - val_accuracy: 0.1000
Epoch 55/200
1/1 [==============================] - ETA: 0s - loss: 1.9475 - accuracy: 0.2308
Epoch 55: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 61ms/step - loss: 1.9475 - accuracy: 0.2308 - val_loss: 4.4981 - val_accuracy: 0.1000
Epoch 56/200
1/1 [==============================] - ETA: 0s - loss: 1.8787 - accuracy: 0.2051
Epoch 56: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 63ms/step - loss: 1.8787 - accuracy: 0.2051 - val_loss: 4.4370 - val_accuracy: 0.1000
Epoch 57/200
1/1 [==============================] - ETA: 0s - loss: 1.8184 - accuracy: 0.2308
Epoch 57: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 61ms/step - loss: 1.8184 - accuracy: 0.2308 - val_loss: 4.3811 - val_accuracy: 0.0000e+00
Epoch 58/200
1/1 [==============================] - ETA: 0

1/1 [==============================] - 0s 63ms/step - loss: 1.6311 - accuracy: 0.1795 - val_loss: 4.3547 - val_accuracy: 0.0000e+00
Epoch 85/200
1/1 [==============================] - ETA: 0s - loss: 1.6278 - accuracy: 0.2051
Epoch 85: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 63ms/step - loss: 1.6278 - accuracy: 0.2051 - val_loss: 4.4035 - val_accuracy: 0.0000e+00
Epoch 86/200
1/1 [==============================] - ETA: 0s - loss: 1.6251 - accuracy: 0.2564
Epoch 86: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 60ms/step - loss: 1.6251 - accuracy: 0.2564 - val_loss: 4.4490 - val_accuracy: 0.0000e+00
Epoch 87/200
1/1 [==============================] - ETA: 0s - loss: 1.6229 - accuracy: 0.2821
Epoch 87: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 59ms/step - loss: 1.6229 - accuracy: 0.2821 - val_loss: 4.4907 - val_accuracy: 0.0000e+00
Epoch 88/200
1/1 [============================

1/1 [==============================] - ETA: 0s - loss: 1.5931 - accuracy: 0.2821
Epoch 114: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 60ms/step - loss: 1.5931 - accuracy: 0.2821 - val_loss: 5.1574 - val_accuracy: 0.0000e+00
Epoch 115/200
1/1 [==============================] - ETA: 0s - loss: 1.5923 - accuracy: 0.2051
Epoch 115: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 61ms/step - loss: 1.5923 - accuracy: 0.2051 - val_loss: 5.1763 - val_accuracy: 0.0000e+00
Epoch 116/200
1/1 [==============================] - ETA: 0s - loss: 1.5916 - accuracy: 0.2564
Epoch 116: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 60ms/step - loss: 1.5916 - accuracy: 0.2564 - val_loss: 5.1928 - val_accuracy: 0.0000e+00
Epoch 117/200
1/1 [==============================] - ETA: 0s - loss: 1.5910 - accuracy: 0.2821
Epoch 117: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 

Epoch 144/200
1/1 [==============================] - ETA: 0s - loss: 1.5752 - accuracy: 0.2821
Epoch 144: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 61ms/step - loss: 1.5752 - accuracy: 0.2821 - val_loss: 5.7529 - val_accuracy: 0.0000e+00
Epoch 145/200
1/1 [==============================] - ETA: 0s - loss: 1.5750 - accuracy: 0.2821
Epoch 145: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 60ms/step - loss: 1.5750 - accuracy: 0.2821 - val_loss: 5.7732 - val_accuracy: 0.0000e+00
Epoch 146/200
1/1 [==============================] - ETA: 0s - loss: 1.5748 - accuracy: 0.2821
Epoch 146: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 60ms/step - loss: 1.5748 - accuracy: 0.2821 - val_loss: 5.7937 - val_accuracy: 0.0000e+00
Epoch 147/200
1/1 [==============================] - ETA: 0s - loss: 1.5743 - accuracy: 0.2564
Epoch 147: val_loss did not improve from 2.02505
1/1 [=======================

Epoch 174/200
1/1 [==============================] - ETA: 0s - loss: 1.5619 - accuracy: 0.2564
Epoch 174: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 62ms/step - loss: 1.5619 - accuracy: 0.2564 - val_loss: 6.2560 - val_accuracy: 0.0000e+00
Epoch 175/200
1/1 [==============================] - ETA: 0s - loss: 1.5615 - accuracy: 0.2821
Epoch 175: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 60ms/step - loss: 1.5615 - accuracy: 0.2821 - val_loss: 6.2703 - val_accuracy: 0.0000e+00
Epoch 176/200
1/1 [==============================] - ETA: 0s - loss: 1.5611 - accuracy: 0.2821
Epoch 176: val_loss did not improve from 2.02505
1/1 [==============================] - 0s 62ms/step - loss: 1.5611 - accuracy: 0.2821 - val_loss: 6.2851 - val_accuracy: 0.0000e+00
Epoch 177/200
1/1 [==============================] - ETA: 0s - loss: 1.5607 - accuracy: 0.2564
Epoch 177: val_loss did not improve from 2.02505
1/1 [=======================

ValueError: Data cardinality is ambiguous:
  x sizes: 16, 16, 16, 16, 16
  y sizes: 49
Make sure all arrays contain the same number of samples.

In [ ]:
def prepare_single_video(frames, skeletons, hands, arm_angles):
    num_frames = frames.shape[1]
    video_length = min(MAX_SEQ_LENGTH, num_frames)

    frame_mask = np.zeros((1, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros((1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float16")
    frame_skeletons = np.zeros((1, MAX_SEQ_LENGTH, SKELETON_FEATURES), dtype="float16")
    frame_hands = np.zeros((1, MAX_SEQ_LENGTH, HAND_FEATURES), dtype="float16")
    frame_images = np.zeros((1, MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3), dtype="float16")
    frame_arm_angles = np.zeros((1, MAX_SEQ_LENGTH, 2), dtype="float16")  # 2는 팔 각도 데이터 차원
    
    feature_extractor = Sequential([
        ResNet50(include_top=False, weights='imagenet', pooling='avg'),  # 예시로 평균 풀링 사용
        Dense(NUM_FEATURES, activation='relu')  # NUM_FEATURES에 맞는 덴스 레이어 추가
    ])
    
    for j in range(video_length):
        # 이미지 데이터 전처리 및 특징 추출
        image_feature = preprocess_image(frames[j:j+1])
        image_feature = np.expand_dims(image_feature, axis=0)
        feature_result = feature_extractor.predict(image_feature)

        # frame_images 대신 feature_result를 사용
        frame_features[0, j] = feature_result

        # Mediapipe 데이터 전처리
        skeleton_feature = preprocess_skeleton_data(skeletons[j])
        hand_feature = preprocess_hand_data(hands[j])        
        arm_angle_feature = np.array(arm_angles[j])

        frame_skeletons[0, j] = skeleton_feature
        frame_hands[0, j] = hand_feature
        frame_mask[0, j] = 1        
        frame_arm_angles[0, j] = arm_angle_feature

    return frame_features, frame_skeletons, frame_hands, frame_mask, frame_images, frame_arm_angles

def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()
    frames, skeletons, hands, arm_angles = load_video(path)    
    try:
        num_frames = frames.shape[1]
    except IndexError:
        print("Error: Unable to determine the number of frames. Frames shape:", frames.shape)
        return None
    
    frame_features, frame_skeletons, frame_hands, frame_mask, frame_images, frame_arm_angles = prepare_single_video(frames, skeletons, hands, arm_angles)
    probabilities = sequence_model.predict([frame_features, frame_skeletons, frame_hands,frame_arm_angles, frame_mask])[0]
    
    for i in np.argsort(probabilities)[::-1]:
        print(f"{class_vocab[i]} : {probabilities[i] * 100:5.2f}%")
    
    return frame_images

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)